# Pygame 2

### Изображения

In [ ]:
# Для работы с изображениями существует модуль image https://www.pygame.org/docs/ref/image.html
# Сами изображения являются не отдельным классом, а представлены как Surface
# Пример ниже создает холст с изображением. 
image = pygame.image.load("picture_name")

In [ ]:
def load_image(name, color_key=None):
    fullname = os.path.join('data', name)
    try:
        image = pygame.image.load(fullname).convert()
    except pygame.error as message:
        print('Cannot load image:', name)
        raise SystemExit(message)

    if color_key is not None:
        if color_key == -1:
            color_key = image.get_at((0, 0))
        image.set_colorkey(color_key)
    else:
        image = image.convert_alpha()
    return image

In [ ]:
image = pygame.transform.scale(image, (200, 100))
# Изображение можно масштабировать
# Подробнее о данном модуле можно прочитать в документации https://www.pygame.org/docs/ref/transform.html

### Спрайты, анимация спрайтов

In [ ]:
# Спрайт - произвольный игровой графический объект 
# Необходимые параметры - image и rect, где image - Surface, a rect - прямоугольний, ограничивающий загруженное изображение
# Для работы со спрайтами существует специальный объект sprite (https://www.pygame.org/docs/ref/sprite.html)
# Особенность работы со спрайтами в том, что у них нет функции draw
# для того, чтобы их отрисовать их нужно объединить в группу спрайтов и вызвать у группы метод draw

# создадим группу, содержащую все спрайты
all_sprites = pygame.sprite.Group()
# создадим спрайт
sprite = pygame.sprite.Sprite()
# определим его вид
sprite.image = load_image("star.png")
# и размеры
sprite.rect = sprite.image.get_rect()
# добавим спрайт в группу
all_sprites.add(sprite)
sprite.rect.x = 5
sprite.rect.y = 20

# в главном игровом цикле
all_sprites.draw(screen)

In [ ]:
# Как видно из примера, работать со спрайтами таким образом не совсем удобно
# Работа станет значительно проще, если создать отдельный класс для графического объета и унаследовать его от спрайта
class Star(pg.sprite.Sprite):

    def __init__(self, *group):
        super().__init__(*group)

        self.len = randint(1, 6)

        self.image = pg.Surface((self.len, self.len))
        pg.draw.circle(self.image, (200, 200, 200), (self.len // 2, self.len // 2), self.len // 2)

        self.rect = self.image.get_rect()
        self.x = self.rect.x = randrange(width)
        self.y = self.rect.y = randrange(height)

    def update(self):
        self.x += 0.1 * (self.len // 2)
        self.y += 0.05
        self.rect.x = self.x % width
        self.rect.y = self.y % height

In [ ]:
# Инициализация до игрового цикла
all_sprites = pg.sprite.Group()
for _ in range(70):
    Star(all_sprites)
    
# Отрисовка в игровом цикле
all_sprites.draw(screen)
all_sprites.update()

<img src="https://i.ibb.co/x7fjSRV/1.gif" width="450px">

In [ ]:
# Взаимодействовать со спрайтами удобнее через метод update и передавать в него события, нежели по событию переберать все спрайты

def update(self, *args):
    self.x += 0.1 * (self.len // 2)
    self.y += 0.05
    self.rect.x = self.x % width
    self.rect.y = self.y % height

    if args and args[0].type == pg.MOUSEMOTION and self.rect.collidepoint(args[0].pos):
        pg.draw.circle(self.image, pg.Color('yellow'), (self.len // 2, self.len // 2), self.len // 2)

### Анимация спрайтов

In [ ]:
# Для реализации анимации наследуются также от класса sprite
# Идея заключается в том, что в классе хранятся кадры с изображениями и последовательно меняются при взаимодействии
# Спрайты, как правило, хранятся листами, совмещающие в себе несколько последовательных изображений.

class AnimatedSprite(pygame.sprite.Sprite):
    def __init__(self, sheet, columns, rows, x, y):
        super().__init__(all_sprites)
        self.frames = []
        self.cut_sheet(sheet, columns, rows)
        self.cur_frame = 0
        self.image = self.frames[self.cur_frame]
        self.rect = self.rect.move(x, y)

    def cut_sheet(self, sheet, columns, rows):
        self.rect = pygame.Rect(0, 0, sheet.get_width() // columns, 
                                sheet.get_height() // rows)
        for j in range(rows):
            for i in range(columns):
                frame_location = (self.rect.w * i, self.rect.h * j)
                self.frames.append(sheet.subsurface(pygame.Rect(
                    frame_location, self.rect.size)))

    def update(self):
        self.cur_frame = (self.cur_frame + 1) % len(self.frames)
        self.image = self.frames[self.cur_frame]
        
# Для того, чтобы анимация работала независимо от игрового времени и не не тормозила всю игру
# можно ввести в классе счетчик итераций и менять изображение, скажем, каждую пятую итерацию.

<img src="https://i.ibb.co/9cbSnDg/pygame-8-1.png" width="650px">
<img src="https://i.ibb.co/RpWHsNG/pygame-8-2.gif" width="150px">

### Столкновения

In [ ]:
# Проверить спрайты на столкновение можно двумя способами:
# 1. По ограничивающему прямоугольнику (метод collide_rect())
# 2. По ограничивающей окружности (метод collide_circle())
# В оба метода передаются спрайты, результат - True либо False

# Более общий случай - столкновение группы спрайтов
# Реализация проверки - в методе update каждый спрайт будет сравниваться с какой-либо группой
# Рассмотрим на примере столкновения шаров со стенками
# Для этого создадим класс для шара и стенки

class Ball(pygame.sprite.Sprite):
    def __init__(self, radius, x, y):
        super().__init__(all_sprites)
        self.radius = radius
        self.image = pygame.Surface((2 * radius, 2 * radius),
                                    pygame.SRCALPHA, 32)
        pygame.draw.circle(self.image, pygame.Color("red"),
                           (radius, radius), radius)
        self.rect = pygame.Rect(x, y, 2 * radius, 2 * radius)
        self.vx = random.randint(-5, 5)
        self.vy = random.randrange(-5, 5)

    def update(self):
        self.rect = self.rect.move(self.vx, self.vy)
        if pygame.sprite.spritecollideany(self, horizontal_borders):
            self.vy = -self.vy
        if pygame.sprite.spritecollideany(self, vertical_borders):
            self.vx = -self.vx
            
# Функция spritecollideany() возвращает спрайт из группы, с которым произошло столкновение или None, 
# если столкновение не обнаружено.
# Другая функция, spritecollide(), принимает в качестве аргументов так же спрайт и группу — возвращает список 
# спрайтов из группы, с которыми произошло пересечение. 
# Третьим параметром можно передать логическое значение True, и тогда все спрайты, с которыми есть пересечение, 
# будут уничтожены и убраны из группы.
        
horizontal_borders = pygame.sprite.Group()
vertical_borders = pygame.sprite.Group()
class Border(pygame.sprite.Sprite):
    # строго вертикальный или строго горизонтальный отрезок
    def __init__(self, x1, y1, x2, y2):
        super().__init__(all_sprites)
        if x1 == x2:  # вертикальная стенка
            self.add(vertical_borders)
            self.image = pygame.Surface([1, y2 - y1])
            self.rect = pygame.Rect(x1, y1, 1, y2 - y1)
        else:  # горизонтальная стенка
            self.add(horizontal_borders)
            self.image = pygame.Surface([x2 - x1, 1])
            self.rect = pygame.Rect(x1, y1, x2 - x1, 1)


In [ ]:
Border(5, 5, width - 5, 5)
Border(5, height - 5, width - 5, height - 5)
Border(5, 5, 5, height - 5)
Border(width - 5, 5, width - 5, height - 5)

for i in range(10):
    Ball(20, 100, 100)

<img src="https://i.ibb.co/0cjXw1C/2021-07-22-17-09-41.gif" width="300px">

In [ ]:
# Пересечение со сложными объектами сравниваются по маске с помощью метода pygame.sprite.collide_mask()
# Для этого необходимо в конструкторе класса вычислить маску 
# Пр. self.mask = pygame.mask.from_surface(self.image)

### Заставка и экран конца игры

In [ ]:
# В большинстве случаев, заставка и экран конца игры это еще один игровой цикл, реализованный как отдельная функция
def terminate():
    pygame.quit()
    sys.exit()

def start_screen():
    intro_text = ["ЗАСТАВКА", "",
                  "Правила игры",
                  "Если в правилах несколько строк,",
                  "приходится выводить их построчно"]

    fon = pygame.transform.scale(load_image('fon.jpg'), (WIDTH, HEIGHT))
    screen.blit(fon, (0, 0))
    font = pygame.font.Font(None, 30)
    text_coord = 50
    for line in intro_text:
        string_rendered = font.render(line, 1, pygame.Color('white'))
        intro_rect = string_rendered.get_rect()
        text_coord += 10
        intro_rect.top = text_coord
        intro_rect.x = 10
        text_coord += intro_rect.height
        screen.blit(string_rendered, intro_rect)

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                terminate()
            elif event.type == pygame.KEYDOWN or \
                    event.type == pygame.MOUSEBUTTONDOWN:
                return  # начинаем игру
        pygame.display.flip()
        clock.tick(FPS)

### Уровни игры

In [ ]:
# Уровни в игре удобно хранить в текстовых файлах. Так их очень удобно редактировать. 

...###
..##.#.####
.##..###..#
##........#
#...@..#..#
###..###..#
..#..#....#
.##.##.#.##
.#......##
.#.....##
.#######

In [ ]:
def load_level(filename):
    filename = "data/" + filename
    # читаем уровень, убирая символы перевода строки
    with open(filename, 'r') as mapFile:
        level_map = [line.strip() for line in mapFile]

    # и подсчитываем максимальную длину    
    max_width = max(map(len, level_map))

    # дополняем каждую строку пустыми клетками ('.')    
    return list(map(lambda x: x.ljust(max_width, '.'), level_map))


In [ ]:
# Перед отрисовкой уровня создадим классы клеток на доске
# Статичные клетки для удобства поместим в словарь

tile_images = {
    'wall': load_image('box.png'),
    'empty': load_image('grass.png')
}
player_image = load_image('mario.png')

tile_width = tile_height = 50


class Tile(pygame.sprite.Sprite):
    def __init__(self, tile_type, pos_x, pos_y):
        super().__init__(tiles_group, all_sprites)
        self.image = tile_images[tile_type]
        self.rect = self.image.get_rect().move(
            tile_width * pos_x, tile_height * pos_y)


class Player(pygame.sprite.Sprite):
    def __init__(self, pos_x, pos_y):
        super().__init__(player_group, all_sprites)
        self.image = player_image
        self.rect = self.image.get_rect().move(
            tile_width * pos_x + 15, tile_height * pos_y + 5)

In [ ]:
# основной персонаж
player = None

# группы спрайтов
all_sprites = pygame.sprite.Group()
tiles_group = pygame.sprite.Group()
player_group = pygame.sprite.Group()

def generate_level(level):
    new_player, x, y = None, None, None
    for y in range(len(level)):
        for x in range(len(level[y])):
            if level[y][x] == '.':
                Tile('empty', x, y)
            elif level[y][x] == '#':
                Tile('wall', x, y)
            elif level[y][x] == '@':
                Tile('empty', x, y)
                new_player = Player(x, y)
    # вернем игрока, а также размер поля в клетках            
    return new_player, x, y

# Теперь для загрузки и отрисовки достаточно выполнить 
player, level_x, level_y = generate_level(load_level('map.txt'))

<img src="https://i.ibb.co/C2k0nst/2021-07-23-10-00-26.png" width="500px">